In [22]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси!

In [23]:
df = pd.read_csv('taxi_dataset_with_target.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [24]:
from sklearn.linear_model import LinearRegression

X = df.drop(columns=['trip_duration', 'pickup_datetime'], axis = 1)
y = df['trip_duration']

In [25]:
model=LinearRegression()
model.fit(X, y)

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_**

In [26]:
model.coef_

array([198.46313674,   0.29631295,  56.46912165, 115.27353763])

In [27]:
model.intercept_

np.float64(171.65658145580664)

Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [53]:
import numpy as np

### Your code is here

def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
        """
        :param X: матрица объектов
        :param Y: вектор (матрица с 1 столбцом) ответов
        :param fit_intercept: добавить ли константную переменную в данные?

        :return: одномерный numpy-array с полученными коэффициентами beta
        """

        if fit_intercept:
          b = np.linalg.inv((np.concat([X, np.ones((X.shape[0],1))], axis=1).T @ np.concat([X, np.ones((X.shape[0],1))], axis=1))) @ np.concat([X, np.ones((X.shape[0],1))], axis=1).T @ Y
        else:
          b = np.linalg.inv(X.T @ X) @ X.T @ Y
        return b

In [54]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

LinearRegressionByMatrix(X_train, Y_train)

array([198.46313674,   0.29631295,  56.46912165, 115.27353763,
       171.65658146])

Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)